# Eliza in Python (1)

본 내용은 The 11th International Conference on Computational Semantics(IWCS 2015)의 부대 행사인 Computational Semantics Hackathon의 자료 중 Dialogue system 파트를 번역한 자료입니다([출처](https://github.com/iwcs15-hack/dialog_system)).

Based on [this source code](https://www.daniweb.com/programming/software-development/code/380743/eliza-aka-therapist-facelift)

In [117]:
import random
import re

Eliza는 사용자가 말한 내용을 발화의 인칭을 바꾸어 말함으로써(ex. I 와 you를 맞바꾸는 등), 사용자의 말을 흉내낸다. 여기서는 간단한 규칙만으로 이를 구현해 볼 것이다. 보다 정교하게 만들기 위해 NLTK를 사용할 수 있지만, 정교한 방법은 다음편에 다룰 것이다.

아래 셀의 `reflection_of`는 key에 해당하는 단어를 이를 이에 대한 대체어로 대응시키는 dictionary이다. `translate` 함수는 매칭된 패턴을 입력받아 이를 정규표현식을 이용해 단어로 분할한 뒤, 이에 대해 프로그램이 찾은 반영(reflection)으로 대체한다.

In [2]:

reflection_of = {
    "am"    : "are",
    "was"   : "were",
    "i"     : "you",
    "i'd"   : "you would",
    "i've"  : "you have",
    "i'll"  : "you will",
    "i'm"   : "you are",
    "my"    : "your",
    "are"   : "am",
    "you're": "I am",
    "you've": "I have",
    "you'll": "I will",
    "your"  : "my",
    "yours" : "mine",
    "you"   : "me",
    "me"    : "you" }


def translate(this):
    return ' '.join(reflection_of[word] if word in reflection_of else word for word in re.findall(r"[\w']+", this.lower()))

In [3]:
translate('My father so strict to me')

'your father so strict to you'

In [4]:
translate("You're so kind")

'I am so kind'

Eliza는 Rule-based chatbot으로 다음 규칙들을 기반으로 작동한다.  

    1. 유저 발화 : 유저가 말할 것으로 예상되는 발화로, 만약 실제 유저 발화가 이와 대응되면, 해당 발화에 가능한 응답 후보 리스트가 선택되어 이 중에서 적절한 응답을 선택해 생성할 수 있다.  
    
    2. 가능한 응답 후보 리스트 : therapist(Eliza)는 이 중에서 하나를 랜덤으로 선택한다. 문장 속에 포맷 `%n`이 포함될 수도 있는데, 이것은 유저의 발화에서 1번 규칙의 정규표현식을 이용해 매칭된 패턴 그룹들 중 n번째 그룹에 해당되는 것으로 대체한다는 표현이다. 이 예제의 Eliza는 39가지 규칙으로 이루어져 있으며, 규칙이 더 다양해질수록, 더 복잡한 답변이 가능하다. 

In [5]:
rules = [(re.compile(x[0]), x[1]) for x in [
   ['How are you?',
      [ "I'm fine, thank you."]],
    ["I need (.*)",
    [   "Why do you need %1?",
        "Would it really help you to get %1?",
        "Are you sure you need %1?"]],
    ["Why don't you (.*)",
    [   "Do you really think I don't %1?",
        "Perhaps eventually I will %1.",
        "Do you really want me to %1?"]],
    ["Why can't I (.*)",
    [   "Do you think you should be able to %1?",
        "If you could %1, what would you do?",
        "I don't know -- why can't you %1?",
        "Have you really tried?"]],
    ["I can't (.*)",
    [   "How do you know you can't %1?",
        "Perhaps you could %1 if you tried.",
        "What would it take for you to %1?"]],
    ["I am (.*)",
    [   "Did you come to me because you are %1?",
        "How long have you been %1?",
        "How do you feel about being %1?"]],
    ["I'm (.*)",
    [   "How does being %1 make you feel?",
        "Do you enjoy being %1?",
        "Why do you tell me you're %1?",
        "Why do you think you're %1?"]],
    ["Are you (.*)",
    [   "Why does it matter whether I am %1?",
        "Would you prefer it if I were not %1?",
        "Perhaps you believe I am %1.",
        "I may be %1 -- what do you think?"]],
    ["What (.*)",
    [   "Why do you ask?",
        "How would an answer to that help you?",
        "What do you think?"]],
    ["How (.*)",
    [   "How do you suppose?",
        "Perhaps you can answer your own question.",
        "What is it you're really asking?"]],
    ["Because (.*)",
    [   "Is that the real reason?",
        "What other reasons come to mind?",
        "Does that reason apply to anything else?",
        "If %1, what else must be true?"]],
    ["(.*) sorry (.*)",
    [   "There are many times when no apology is needed.",
        "What feelings do you have when you apologize?"]],
    ["Hello(.*)",
    [   "Hello... I'm glad you could drop by today.",
        "Hi there... how are you today?",
        "Hello, how are you feeling today?"]],
    ["I think (.*)",
    [   "Do you doubt %1?",
        "Do you really think so?",
        "But you're not sure %1?"]],
    ["(.*) friend(.*)",
    [   "Tell me more about your friends.",
        "When you think of a friend, what comes to mind?",
        "Why don't you tell me about a childhood friend?"]],
    ["Yes",
    [   "You seem quite sure.",
        "OK, but can you elaborate a bit?"]],
    ["No",
    [ "Why not?"]],
    ["(.*) computer(.*)",
    [   "Are you really talking about me?",
        "Does it seem strange to talk to a computer?",
        "How do computers make you feel?",
        "Do you feel threatened by computers?"]],
    ["Is it (.*)",
    [   "Do you think it is %1?",
        "Perhaps it's %1 -- what do you think?",
        "If it were %1, what would you do?",
        "It could well be that %1."]],
    ["It is (.*)",
    [   "You seem very certain.",
        "If I told you that it probably isn't %1, what would you feel?"]],
    ["Can you (.*)",
    [   "What makes you think I can't %1?",
        "If I could %1, then what?",
        "Why do you ask if I can %1?"]],
    ["Can I (.*)",
    [   "Perhaps you don't want to %1.",
        "Do you want to be able to %1?",
        "If you could %1, would you?"]],
    ["You are (.*)",
    [   "Why do you think I am %1?",
        "Does it please you to think that I'm %1?",
        "Perhaps you would like me to be %1.",
        "Perhaps you're really talking about yourself?"]],
    ["You're (.*)",
    [   "Why do you say I am %1?",
        "Why do you think I am %1?",
        "Are we talking about you, or me?"]],
    ["I don't (.*)",
    [   "Don't you really %1?",
        "Why don't you %1?",
        "Do you want to %1?"]],
    ["I feel (.*)",
    [   "Good, tell me more about these feelings.",
        "Do you often feel %1?",
        "When do you usually feel %1?",
        "When you feel %1, what do you do?"]],
    ["I have (.*)",
    [   "Why do you tell me that you've %1?",
        "Have you really %1?",
        "Now that you have %1, what will you do next?"]],
    ["I would (.*)",
    [   "Could you explain why you would %1?",
        "Why would you %1?",
        "Who else knows that you would %1?"]],
    ["Is there (.*)",
    [   "Do you think there is %1?",
        "It's likely that there is %1.",
        "Would you like there to be %1?"]],
    ["My (.*)",
    [   "I see, your %1.",
        "Why do you say that your %1?",
        "When your %1, how do you feel?"]],
    ["You (.*)",
    [   "We should be discussing you, not me.",
        "Why do you say that about me?",
        "Why do you care whether I %1?"]],
    ["Why (.*)",
    [   "Why don't you tell me the reason why %1?",
        "Why do you think %1?" ]],
    ["I want (.*)",
    [   "What would it mean to you if you got %1?",
        "Why do you want %1?",
        "What would you do if you got %1?",
        "If you got %1, then what would you do?"]],
    ["(.*) mother(.*)",
    [   "Tell me more about your mother.",
        "What was your relationship with your mother like?",
        "How do you feel about your mother?",
        "How does this relate to your feelings today?",
        "Good family relations are important."]],
    ["(.*) father(.*)",
    [   "Tell me more about your father.",
        "How did your father make you feel?",
        "How do you feel about your father?",
        "Does your relationship with your father relate to your feelings today?",
        "Do you have trouble showing affection with your family?"]],
    ["(.*) child(.*)",
    [   "Did you have close friends as a child?",
        "What is your favorite childhood memory?",
        "Do you remember any dreams or nightmares from childhood?",
        "Did the other children sometimes tease you?",
        "How do you think your childhood experiences relate to your feelings today?"]],
    ["(.*)\?",
    [   "Why do you ask that?",
        "Please consider whether you can answer your own question.",
        "Perhaps the answer lies within yourself?",
        "Why don't you tell me?"]],
    ["quit",
    [   "Thank you for talking with me.",
        "Good-bye.",
        "Thank you, that will be $150.  Have a good day!"]],
  ["(.*)",
  [   "Please tell me more.",
      "Let's change focus a bit... Tell me about your family.",
      "Can you elaborate on that?",
      "Why do you say that %1?",
      "I see.",
      "Very interesting.",
      "%1.",
      "I see.  And what does that tell you?",
      "How does that make you feel?",
      "How do you feel when you say that?"]]
]]

유저로부터 문장이 주어지면, 이 함수는 모든 규칙을 체크하여 가장 만저 매칭되는 규칙을 택한다. 그 다음 이에 대한 응답 후보 중 하나를 랜덤하게 선택하고, %n 항을 수정하여 결과를 반환한다.

In [6]:
def respond(sentence):
    # find a match among keys, last one is quaranteed to match
    for rule, value in rules:
        match = rule.search(sentence)
        if match is not None:
            # found a match ... stuff with corresponding value
            # chosen randomly from among the available options
            resp = random.choice(value)
            # we've got a response... stuff in reflected text where indicated

            while '%' in resp:
                pos = resp.find('%')
                num = int(resp[pos+1:pos+2])
                resp = resp.replace(resp[pos:pos+2], translate(match.group(num)))
            return resp

만약 shell에서 작동하는 interactive version을 만들고자 한다면, 아래 코드를 추가하면 된다.
```python
if __name__ == '__main__':
    print("""
    Therapist
    ---------
    Talk to the program by typing in plain English, using normal upper-
    and lower-case letters and punctuation.  Enter "quit" when done.'""")
    print('='*72)
    print("Hello.  How are you feeling today?")
    s = ""
    while s != "quit":
        s = input(">")
        while s and s[-1] in "!.":
            s = s[:-1]
        print(respond(s))
```


Eliza가 어떻게 작동하는지 `respond` 함수에 대한 몇 가지 예시를 통해 살펴보자

In [10]:
respond('My mother hates me')

'When your mother hates you, how do you feel?'

In [8]:
respond('How are you?')

"I'm fine, thank you."

In [9]:
respond("I need a break.")

'Would it really help you to get a break?'

`translate` 함수를 테스트하다보면 몇 가지 버그 케이스가 보이는데, 이는 이 함수가 사용자 발화에서의 각 단어의 문법적 역할을 제대로 판별하지 못하기 때문이다. 예를 들어 `you`가 문장에서 주어인지 목적어인지, 또는 `you` 또는 `they`가 동사 `were`의 주어인지 아닌지를 제대로 판별하지 못한다. 이러한 유형의 문제를 해결하기 위해서는 단순 정규 표현식을 이용한 방식보다 실질적인 AI가 필요하다.

In [90]:
# 제대로 처리되었을 떄
translate('my mother hates me')

'your mother hates you'

In [91]:
# 두 번째 you가 종속절 문장의 주어지만 목적어로 잘못 파악함
translate('I will do anything you ask')

'you will do anything me ask'

In [94]:
# 제대로 처리되었을 떄
translate('the dogs were crazy')

'the dogs were crazy'

In [95]:
# 3인칭 was를 (1인칭 was로 잘못 파악해) 2인칭으로 were로 변환
translate('the dog was crazy')

'the dog were crazy'

In [12]:
respond("My dog was crazy.")

'Why do you say that your dog were crazy?'

In [113]:
respond("I was crazy")

'you were crazy.'

In [116]:
respond("You said Fred was crazy.")

'Why do you care whether I said fred were crazy?'

In [33]:
respond("I asked you.")

'Why do you say that you asked me?'